In [18]:
# Dependencies
import requests
import pandas as pd
import json
import datetime
import requests
import calendar
import openpyxl
import seaborn as sns
from datetime import datetime, timedelta

from config import grant_type, client_id, client_secret, redirect_url,refresh_token

In [ ]:
# Step 1: Create the OAuth link and authorize the access
oauth_link = f"https://hotels.cloudbeds.com/api/v1.1/oauth?client_id={client_id}&redirect_uri={redirect_url}&response_type=code"
print(f"Please go to this URL to authorize access: {oauth_link}")

# Step 2: Retrieve the authorization code from the redirect URL
auth_code = input("Enter the authorization code from the redirect URL: ")

url = 'https://hotels.cloudbeds.com/api/v1.1/access_token'
headers = {'Content-type': 'application/json'}

# Specify the grant type and other parameters based on your authentication flow
if grant_type == 'authorization_code':
    data = {
        'grant_type': 'authorization_code',
        'client_id': client_id,
        'client_secret': client_secret,
        'redirect_uri': redirect_url,
        'code': auth_code
    }
elif grant_type == 'refresh_token':
    data = {
        'grant_type': 'refresh_token',
        'client_id': client_id,
        'client_secret': client_secret,
        'refresh_token': refresh_token
    }

response = requests.post(url, data=data)

if response.ok:
    access_token_data = response.json()
    access_token = access_token_data["access_token"]
    token_type = access_token_data["token_type"]
    expires_in = access_token_data["expires_in"]
    refresh_token = access_token_data.get("refresh_token")
    print(f"Access token: {access_token}")
    print(f"Token type: {token_type}")
    print(f"Expires in: {expires_in} seconds")
    if refresh_token:
        print(f"Refresh token: {refresh_token}")
else:
    print("Error retrieving access token.")
    print(response.json())

In [23]:
#Pull 1 year past data

url = "https://hotels.cloudbeds.com/api/v1.1/getReservationsWithRateDetails"

# Set up pagination parameters
page_number = 1
page_size = 100

results_past=[]

# Set the date range for the previous year
today = datetime.now()
one_year_ago = today - timedelta(days=365)

while True:
    # Parameters for the API request
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        "reservationCheckOutFrom": one_year_ago.strftime("%Y-%m-%d"),
        "reservationCheckOutTo": today.strftime("%Y-%m-%d"),
        "sortByRecent": True,
        "pageNumber": page_number,
        "pageSize": page_size
    }

    # Make the API request
    response = requests.get(url, params=params, headers=headers)

    # Extract the 'data' field from the API response
    api_data = response.json().get('data', [])

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(api_data)

    # Append the DataFrame to the overall results list
    results_past.append(df)

    # Check if there are more results
    if len(df) < page_size:
        break

    # Increment the page number for the next request
    page_number += 1

# Concatenate all the DataFrames into one
df_all_past = pd.concat(results_past, ignore_index=True)

# Print the final DataFrame
print(df_all_past)

      reservationID  isDeleted          dateCreated       dateCreatedUTC  \
0     4415078104293      False  2024-02-16 14:08:57  2024-02-16 20:08:57   
1     4539724472692      False  2024-02-20 11:08:17  2024-02-20 17:08:17   
2     5158741447740      False  2024-01-16 17:44:23  2024-01-16 23:44:23   
3     5061025955891      False  2024-02-20 19:49:50  2024-02-21 01:49:50   
4     2632506888760      False  2024-01-06 22:01:21  2024-01-07 04:01:21   
...             ...        ...                  ...                  ...   
7322  9049776267868      False  2022-10-17 13:17:57  2022-10-17 18:17:57   
7323  8209988726841      False  2022-10-18 01:48:20  2022-10-18 06:48:20   
7324  4480999257826      False  2022-10-04 13:52:19  2022-10-04 18:52:19   
7325  8901269886916      False  2022-09-21 12:43:29  2022-09-21 17:43:29   
7326  7684467571949      False  2022-09-09 04:38:03  2022-09-09 09:38:03   

             dateModified      dateModifiedUTC      status reservationCheckIn  \
0     

In [24]:
#Pull 1 year future data

url = "https://hotels.cloudbeds.com/api/v1.1/getReservationsWithRateDetails"

# Set up pagination parameters
page_number = 1
page_size = 100

results_future=[]

# Set the date range for the previous year
today = datetime.now()
one_year_ahead = today + timedelta(days=365)

while True:
    # Parameters for the API request
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        "reservationCheckOutFrom": today.strftime("%Y-%m-%d"),
        "reservationCheckOutTo": one_year_ahead.strftime("%Y-%m-%d"),
        "sortByRecent": True,
        "pageNumber": page_number,
        "pageSize": page_size
    }


    # Make the API request
    response = requests.get(url, params=params, headers=headers)

    # Extract the 'data' field from the API response
    api_data = response.json().get('data', [])

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(api_data)

    # Append the DataFrame to the overall results list
    results_future.append(df)

    # Check if there are more results
    if len(df) < page_size:
        break

    # Increment the page number for the next request
    page_number += 1

# Concatenate all the DataFrames into one
df_all_future = pd.concat(results_future, ignore_index=True)

# Print the final DataFrame
print(df_all_future)

     reservationID  isDeleted          dateCreated       dateCreatedUTC  \
0    6350237360308      False  2024-02-21 09:28:30  2024-02-21 15:28:30   
1    6745345070217      False  2024-02-21 09:10:58  2024-02-21 15:10:58   
2    1461013426300      False  2024-02-21 05:16:08  2024-02-21 11:16:08   
3    8565246173543      False  2024-02-20 23:26:18  2024-02-21 05:26:18   
4    3426916627719      False  2024-02-21 07:12:09  2024-02-21 13:12:09   
..             ...        ...                  ...                  ...   
738  9869922866964      False  2023-07-14 18:09:42  2023-07-14 23:09:42   
739  8761565681602      False  2023-07-13 00:52:14  2023-07-13 05:52:14   
740  2168238349320      False  2023-07-11 20:58:56  2023-07-12 01:58:56   
741  1239647899309      False  2023-07-10 17:34:11  2023-07-10 22:34:11   
742  7394391108215      False  2023-07-10 12:56:24  2023-07-10 17:56:24   

            dateModified      dateModifiedUTC     status reservationCheckIn  \
0    2024-02-21 09:2

In [25]:
print(f"We received {len(df_all_past)} past reservations.")
print(f"We received {len(df_all_future)} future resservations.")

We received 7327 past reservations.
We received 743 future resservations.


In [26]:
#create cleaning data function
def expand_dataframe(df):
    expanded_rows = []
    
    for index, row in df.iterrows():
        detailed_rates = row['detailedRates']

        if detailed_rates:
            for date, price in detailed_rates.items():
                new_row = row.copy()
                new_row['date'] = date
                new_row['price'] = price
                expanded_rows.append(new_row)

    expanded_df = pd.DataFrame(expanded_rows)
    expanded_df.drop('detailedRates', axis=1, inplace=True)
    
    expanded_df['source'] = expanded_df['source'].apply(lambda x: x['name'])
    expanded_df.rename(columns={'source': 'source_name'}, inplace=True)
    
    expanded_df['adults'] = expanded_df['rooms'].apply(lambda x: x[0]['adults'])
    expanded_df.drop('rooms', axis=1, inplace=True)
    
    expanded_df['date'] = pd.to_datetime(expanded_df['date'])
    expanded_df['day'] = expanded_df['date'].dt.day
    expanded_df['month'] = expanded_df['date'].dt.month
    expanded_df['year'] = expanded_df['date'].dt.year
    
    return expanded_df

#create filter data for 'canceled' function
def filter_canceled_reservations(df):
    return df[df['status'] != 'canceled']

# Filter 'canceled' reservations
df_all_past = filter_canceled_reservations(df_all_past)
df_all_future = filter_canceled_reservations(df_all_future)

# Clean and expand dataframes
expanded_df_past = expand_dataframe(df_all_past[['reservationID', 'source', 'detailedRates', 'rooms']])
expanded_df_future = expand_dataframe(df_all_future[['reservationID', 'source', 'detailedRates', 'rooms']])

In [27]:
expanded_df_future.head()

,reservationID,source_name,date,price,adults,day,month,year
0,6350237360308,Hostelworld,2024-04-03,32.30,1,3,4,2024
1,6745345070217,Website/Booking Engine,2024-02-25,119.00,4,25,2,2024
2,1461013426300,Website/Booking Engine,2024-02-21,37.00,1,21,2,2024
3,8565246173543,Airbnb (API),2024-02-23,46.92,1,23,2,2024
3,8565246173543,Airbnb (API),2024-02-24,46.92,1,24,2,2024


In [28]:
expanded_df_past.head()

,reservationID,source_name,date,price,adults,day,month,year
0,4415078104293,Website/Booking Engine,2024-02-20,37.00,1,20,2,2024
1,4539724472692,Airbnb (API),2024-02-20,37.74,1,20,2,2024
2,5158741447740,Booking.com,2024-02-20,99.00,2,20,2,2024
3,5061025955891,Booking.com,2024-02-20,37.00,1,20,2,2024
4,2632506888760,Booking.com,2024-02-19,37.00,1,19,2,2024


In [29]:
row_count_expanded_df_future = len(expanded_df_future)
print(f"Number of rows in expanded_df_future: {row_count_expanded_df_future}")

Number of rows in expanded_df_future: 1987


In [30]:
###Revenue Calcs

# Drop blank values
expanded_df_past.dropna(subset=['adults'], inplace=True)
expanded_df_future.dropna(subset=['adults'], inplace=True)

# Remove values after today's date for past_rev
expanded_df_past = expanded_df_past[expanded_df_past['date'] <= today]

# Remove values before today's date for future_rev
expanded_df_future = expanded_df_future[expanded_df_future['date'] >= today]

# Create a pivot table with sum of prices
pivot_table_past_rev = pd.pivot_table(expanded_df_past, values='price', index=['year', 'month'], columns='day', aggfunc='sum')
pivot_table_future_rev = pd.pivot_table(expanded_df_future, values='price', index=['year', 'month'], columns='day', aggfunc='sum')

# Reindex the columns to have labels 1-31
pivot_table_past_rev = pivot_table_past_rev.reindex(columns=range(1, 32))
pivot_table_future_rev = pivot_table_future_rev.reindex(columns=range(1, 32))

# Reindex the rows to have labels as month names and sort by year
sorted_months_past_rev = sorted(pivot_table_past_rev.index.get_level_values('month').unique())
sorted_years_past_rev = sorted(pivot_table_past_rev.index.get_level_values('year').unique())

# Create a MultiIndex with sorted_years_future_rev and sorted_months_future_rev
index = pd.MultiIndex.from_product([sorted_years_past_rev, sorted_months_past_rev], names=['Year', 'Month'])

# Reindex the DataFrame with the new MultiIndex
pivot_table_past_rev = pivot_table_past_rev.reindex(index=index)

# Optional: If you want to reset the index and have 'year' and 'month' as regular columns
pivot_table_past_rev.reset_index(inplace=True)

# Reindex the rows to have labels as month names and sort by year
sorted_months_future_rev = sorted(pivot_table_future_rev.index.get_level_values('month').unique())
sorted_years_future_rev = sorted(pivot_table_future_rev.index.get_level_values('year').unique())

# Create a MultiIndex with sorted_years_future_rev and sorted_months_future_rev
index = pd.MultiIndex.from_product([sorted_years_future_rev, sorted_months_future_rev], names=['Year', 'Month'])

# Reindex the DataFrame with the new MultiIndex
pivot_table_future_rev = pivot_table_future_rev.reindex(index=index)

# Optional: If you want to reset the index and have 'year' and 'month' as regular columns
pivot_table_future_rev.reset_index(inplace=True)

# Fill blank cells with 0 in the DataFrames
pivot_table_past_rev.fillna(0, inplace=True)
pivot_table_future_rev.fillna(0, inplace=True)
pivot_table_past_rev = pivot_table_past_rev.loc[(pivot_table_past_rev != 0).sum(axis=1) >= 3]

# Export DataFrames to separate Excel files
pivot_table_past_rev.to_excel('Output/past_rev.xlsx', index=True, header=True, engine='openpyxl')
pivot_table_future_rev.to_excel('Output/future_rev.xlsx', index=False, header=True, engine='openpyxl')

In [31]:
###OCC Calcs

# Convert 'adults' column to numeric values
expanded_df_past['adults'] = pd.to_numeric(expanded_df_past['adults'])
expanded_df_future['adults'] = pd.to_numeric(expanded_df_future['adults'])

# Create a pivot table with sum of adults per day divided by 64
pivot_table_past_occ = pd.pivot_table(expanded_df_past, values='adults', index=['year', 'month'], columns='day', aggfunc='sum') / 60
pivot_table_future_occ = pd.pivot_table(expanded_df_future, values='adults', index=['year', 'month'], columns='day', aggfunc='sum') / 60

# Reindex the columns to have labels 1-31
pivot_table_past_occ = pivot_table_past_occ.reindex(columns=range(1, 32))
pivot_table_future_occ = pivot_table_future_occ.reindex(columns=range(1, 32))

# Reindex the rows to have labels as month names and sort by year
sorted_months_past_occ = sorted(pivot_table_past_occ.index.get_level_values('month').unique())
sorted_years_past_occ = sorted(pivot_table_past_occ.index.get_level_values('year').unique())

# Create a MultiIndex with sorted_years_future_rev and sorted_months_future_rev
index = pd.MultiIndex.from_product([sorted_years_past_occ, sorted_months_past_occ], names=['Year', 'Month'])

# Reindex the DataFrame with the new MultiIndex
pivot_table_past_occ = pivot_table_past_occ.reindex(index=index)

# Optional: If you want to reset the index and have 'year' and 'month' as regular columns
pivot_table_past_occ.reset_index(inplace=True)

# Reindex the rows to have labels as month names and sort by year
sorted_months_future_occ = sorted(pivot_table_future_occ.index.get_level_values('month').unique())
sorted_years_future_occ = sorted(pivot_table_future_occ.index.get_level_values('year').unique())

# Create a MultiIndex with sorted_years_future_rev and sorted_months_future_rev
index = pd.MultiIndex.from_product([sorted_years_future_occ, sorted_months_future_occ], names=['Year', 'Month'])

# Reindex the DataFrame with the new MultiIndex
pivot_table_future_occ = pivot_table_future_occ.reindex(index=index)

# Optional: If you want to reset the index and have 'year' and 'month' as regular columns
pivot_table_future_occ.reset_index(inplace=True)

# Fill blank cells with 0 in the DataFrames
pivot_table_past_occ.fillna(0, inplace=True)
pivot_table_future_occ.fillna(0, inplace=True)
pivot_table_past_occ = pivot_table_past_occ.loc[(pivot_table_past_occ != 0).sum(axis=1) >= 3]

# Export DataFrames to separate Excel files
pivot_table_past_occ.to_excel('Output/past_occ.xlsx', index=True, header=True, engine='openpyxl')
pivot_table_future_occ.to_excel('Output/future_occ.xlsx', index=False, header=True, engine='openpyxl')

In [96]:
# # Convert 'adults' column to numeric values
# expanded_df_past['adults'] = pd.to_numeric(expanded_df_past['adults'])
# expanded_df_future['adults'] = pd.to_numeric(expanded_df_future['adults'])

# # Create a pivot table with sum of adults per day divided by 64
# pivot_table_past_occ = pd.pivot_table(expanded_df_past, values='adults', index='month', columns='day', aggfunc='sum') / 64
# pivot_table_future_occ = pd.pivot_table(expanded_df_future, values='adults', index='month', columns='day', aggfunc='sum') / 64

# # Reindex the columns to have labels 1-31
# pivot_table_past_occ = pivot_table_past_occ.reindex(columns=range(1, 32))
# pivot_table_future_occ = pivot_table_future_occ.reindex(columns=range(1, 32))

# # Reindex the rows to have labels as month names and sort by year
# sorted_months_past_occ = sorted(pivot_table_past_occ.index.unique(), key=lambda x: (expanded_df_past[expanded_df_past['month'] == x]['year'].min(), x))
# pivot_table_past_occ = pivot_table_past_occ.reindex(index=sorted_months_past_occ)
# pivot_table_past_occ.index = pivot_table_past_occ.index.map(lambda x: calendar.month_abbr[x])

# sorted_months_future_occ = sorted(pivot_table_future_occ.index.unique(), key=lambda x: (expanded_df_future[expanded_df_future['month'] == x]['year'].min(), x))
# pivot_table_future_occ = pivot_table_future_occ.reindex(index=sorted_months_future_occ)
# pivot_table_future_occ.index = pivot_table_future_occ.index.map(lambda x: calendar.month_abbr[x])

# # Fill blank cells with 0 in the DataFrames
# pivot_table_past_occ.fillna(0, inplace=True)
# pivot_table_future_occ.fillna(0, inplace=True)
# pivot_table_past_occ = pivot_table_past_occ.loc[(pivot_table_past_rev != 0).sum(axis=1) >= 3]

# # Export DataFrames to separate Excel files
# pivot_table_past_occ.to_excel('Output/past_occ.xlsx', index=True, header=True, engine='openpyxl')
# pivot_table_future_occ.to_excel('Output/future_occ.xlsx', index=True, header=True, engine='openpyxl')


In [32]:
###ADR Calcs

# Assuming your DataFrame has DateTimeIndex, if not, replace it with your actual index
date_index = pivot_table_past_rev.index

# Calculate ADR for past_rev excluding the first two columns
pivot_table_past_adr = pivot_table_past_rev.iloc[:, 2:].div(64).div(pivot_table_past_occ.iloc[:, 2:])

# Assuming your DataFrame has DateTimeIndex, if not, replace it with your actual index
date_index_future = pivot_table_future_rev.index

# Calculate ADR for future_rev excluding the first two columns
pivot_table_future_adr = pivot_table_future_rev.iloc[:, 2:].div(64).div(pivot_table_future_occ.iloc[:, 2:])

# Fill blank cells with 0 in the DataFrames
pivot_table_past_adr.fillna(0, inplace=True)
pivot_table_future_adr.fillna(0, inplace=True)

# Add the first two columns back to the calculated DataFrames
pivot_table_past_adr = pd.concat([pivot_table_past_rev.iloc[:, :2], pivot_table_past_adr], axis=1)
pivot_table_future_adr = pd.concat([pivot_table_future_rev.iloc[:, :2], pivot_table_future_adr], axis=1)

# Export styled DataFrames to separate Excel files
pivot_table_past_adr.set_index(date_index).to_excel('Output/past_adr.xlsx', index=True, header=True, engine='openpyxl')
pivot_table_future_adr.set_index(date_index_future).to_excel('Output/future_adr.xlsx', index=True, header=True, engine='openpyxl')


In [94]:
# # Calculate ADR for past_rev
# pivot_table_past_adr = pivot_table_past_rev / 64 / (pivot_table_past_occ)

# # Calculate ADR for future_rev
# pivot_table_future_adr = pivot_table_future_rev / 64 / (pivot_table_future_occ)

# # Fill blank cells with 0 in the DataFrames
# pivot_table_past_adr.fillna(0, inplace=True)
# pivot_table_future_adr.fillna(0, inplace=True)

# # Export styled DataFrames to separate Excel files
# pivot_table_past_adr.to_excel('Output/past_adr.xlsx', index=True, header=True, engine='openpyxl')
# pivot_table_future_adr.to_excel('Output/future_adr.xlsx', index=True, header=True, engine='openpyxl')


In [33]:
###RevPAB Calcs

# Assuming your DataFrame has DateTimeIndex, if not, replace it with your actual index
date_index = pivot_table_past_rev.index

# Calculate ADR for past_rev excluding the first two columns
pivot_table_past_revpab = pivot_table_past_rev.iloc[:, 2:].div(64).mul(pivot_table_past_occ.iloc[:, 2:])

# Assuming your DataFrame has DateTimeIndex, if not, replace it with your actual index
date_index_future = pivot_table_future_rev.index

# Calculate ADR for future_rev excluding the first two columns
pivot_table_future_revpab = pivot_table_future_rev.iloc[:, 2:].div(64).mul(pivot_table_future_occ.iloc[:, 2:])

# Fill blank cells with 0 in the DataFrames
pivot_table_past_revpab.fillna(0, inplace=True)
pivot_table_future_revpab.fillna(0, inplace=True)

# Add the first two columns back to the calculated DataFrames
pivot_table_past_revpab = pd.concat([pivot_table_past_rev.iloc[:, :2], pivot_table_past_revpab], axis=1)
pivot_table_future_revpab = pd.concat([pivot_table_future_rev.iloc[:, :2], pivot_table_future_revpab], axis=1)

# Export styled DataFrames to separate Excel files
pivot_table_past_revpab.set_index(date_index).to_excel('Output/past_revpab.xlsx', index=True, header=True, engine='openpyxl')
pivot_table_future_revpab.set_index(date_index_future).to_excel('Output/future_revpab.xlsx', index=True, header=True, engine='openpyxl')

In [34]:
unique_values = expanded_df_past['source_name'].unique().tolist()
unique_values


['Website/Booking Engine',
 'Airbnb (API)',
 'Booking.com',
 'Hostelworld',
 'Expedia',
 'HotelTonight',
 'Email',
 'Phone']

In [43]:
# Assuming 'Website/Booking Engine', 'Email', 'Phone' are the values you want to count
values_to_count = ['Website/Booking Engine', 'Email', 'Phone']

# Create a new column with 1 for specified values and 0 otherwise, multiplied by 'adults'
expanded_df_past['count_column'] = (expanded_df_past['source_name'].isin(values_to_count)).astype(int) * expanded_df_past['adults']

# Group by 'year', 'month', and 'day' and sum the 'count_column' and 'adults'
grouped_data_past = expanded_df_past.groupby(['year', 'month', 'day']).agg({'count_column': 'sum', 'adults': 'sum'})

# Calculate the ratio by dividing 'count_column' by 'adults'
grouped_data_past['ratio'] = grouped_data_past['count_column'] / grouped_data_past['adults']

# Pivot the DataFrame to have 'day' as columns, 'year' and 'month' as indices
pivot_table_past_book = grouped_data_past.pivot_table(index=['year', 'month'], columns='day', values='ratio', fill_value=0)

# Create a new column with 1 for specified values and 0 otherwise, multiplied by 'adults'
expanded_df_future['count_column'] = (expanded_df_future['source_name'].isin(values_to_count)).astype(int) * expanded_df_future['adults']

# Group by 'year', 'month', and 'day' and sum the 'count_column' and 'adults'
grouped_data_future = expanded_df_future.groupby(['year', 'month', 'day']).agg({'count_column': 'sum', 'adults': 'sum'})

# Calculate the ratio by dividing 'count_column' by 'adults'
grouped_data_future['ratio'] = grouped_data_future['count_column'] / grouped_data_future['adults']

# Pivot the DataFrame to have 'day' as columns, 'year' and 'month' as indices
pivot_table_future_book = grouped_data_future.pivot_table(index=['year', 'month'], columns='day', values='ratio', fill_value=0)

# Reindex the rows to have labels as month names and sort by year
sorted_months_past_book = sorted(pivot_table_past_book.index.get_level_values('month').unique())
sorted_years_past_book = sorted(pivot_table_past_book.index.get_level_values('year').unique())

# Create a MultiIndex with sorted_years_future_rev and sorted_months_future_rev
index = pd.MultiIndex.from_product([sorted_years_past_book, sorted_months_past_book], names=['Year', 'Month'])

# Reindex the DataFrame with the new MultiIndex
pivot_table_past_book = pivot_table_past_book.reindex(index=index)

# Optional: If you want to reset the index and have 'year' and 'month' as regular columns
pivot_table_past_book.reset_index(inplace=True)

# Reindex the rows to have labels as month names and sort by year
sorted_months_future_book = sorted(pivot_table_future_book.index.get_level_values('month').unique())
sorted_years_future_book = sorted(pivot_table_future_book.index.get_level_values('year').unique())

# Create a MultiIndex with sorted_years_future_rev and sorted_months_future_rev
index = pd.MultiIndex.from_product([sorted_years_future_book, sorted_months_future_book], names=['Year', 'Month'])

# Reindex the DataFrame with the new MultiIndex
pivot_table_future_book = pivot_table_future_book.reindex(index=index)

# Optional: If you want to reset the index and have 'year' and 'month' as regular columns
pivot_table_future_book.reset_index(inplace=True)

# Fill blank cells with 0 in the DataFrames
pivot_table_past_book.fillna(0, inplace=True)
pivot_table_future_book.fillna(0, inplace=True)
pivot_table_past_book = pivot_table_past_book.loc[(pivot_table_past_book != 0).sum(axis=1) >= 3]

# Export DataFrames to separate Excel files
pivot_table_past_book.to_excel('Output/past_book.xlsx', index=True, header=True, engine='openpyxl')
pivot_table_future_book.to_excel('Output/future_book.xlsx', index=True, header=True, engine='openpyxl')


In [ ]:
### Organic Bookings



In [95]:
# # Calculate RevPAB for past_rev
# pivot_table_past_revpab = pivot_table_past_rev / 64 * pivot_table_past_occ

# # Calculate RevPAB for future_rev
# pivot_table_future_revpab = pivot_table_future_rev / 64 * pivot_table_future_occ

# # Fill blank cells with 0 in the DataFrames
# pivot_table_past_revpab.fillna(0, inplace=True)
# pivot_table_future_revpab.fillna(0, inplace=True)

# # Export styled DataFrames to separate Excel files
# pivot_table_past_revpab.to_excel('Output/past_revpab.xlsx', index=True, header=True, engine='openpyxl')
# pivot_table_future_revpab.to_excel('Output/future_revpab.xlsx', index=True, header=True, engine='openpyxl')


In [30]:
# pivot_table_past_rev.to_csv('Output/past_rev.csv', index=True)
# pivot_table_future_rev.to_csv('Output/future_rev.csv', index=True)
# pivot_table_past_occ.to_csv('Output/past_occ.csv', index=True)
# pivot_table_future_occ.to_csv('Output/future_occ.csv', index=True)
